# Итоговый проект (урок № 9), Кривоногов Н.В.

## Step 2: PREDICT

### Проверка работоспособности и качества пайплайна

Здесь я еще не запускаю никакое API, а загружаю модель (pipeline) напрямую и проверяю на отложенной (тестовой) выборке

In [1]:
import pandas as pd
import numpy as np
import dill
from sklearn.metrics import precision_recall_curve

In [2]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [3]:
X_test.head(3)

,area,perimeter,majoraxis,minoraxis,eccentricity,convex_area,extent,class
0,12442,459.535004,187.508850,87.187302,0.885323,12941,0.587580,1
1,12408,437.014008,179.741165,88.829605,0.869343,12598,0.636928,0
2,12867,449.079987,181.700561,91.341064,0.864460,13152,0.649062,0


In [4]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [5]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('convex_area',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='convex_area')),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('eccentricity',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='eccentricity')),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('perimeter',
                                                 Pipeline(steps=[('selector',
             

In [6]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [7]:
preds[:10]

array([0.282148  , 0.02923144, 0.05504274, 0.91505585, 0.00907615,
       0.67723118, 0.00793627, 0.90827513, 0.99296705, 0.63115487])

In [8]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.4469906062611084, F-Score=0.929, Precision=0.931, Recall=0.926
